In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Echarts Jupyter lab support
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [ ]:
# Echarts imports
from pyecharts import options as opts
from pyecharts.globals import  ThemeType
from pyecharts.charts import Line

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt

In [ ]:
from common.log_helper import LogHelper
from backtest.scale_backtest_args import ScaleBacktestArgGen
from backtest.scale_backtest import ScaleBacktest
from modelservice.alpha_consts import SignalDirection
from notebooks.echart_bt_plots import *
from notebooks.echart_df_plot import *

import requests
import json
import logging
import sys
import pandas as pd

In [ ]:
#LogHelper.configure_logging(verbose=False)
pd.set_option("display.precision", 2)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
def colorize(val):
    if val == 'B':
        color = 'green'
    elif val == 'S':
        color = 'red'
    else:
        color = 'black'
    return 'color: %s' % color

In [ ]:
BtArgs = {
    'common': {
        'start_time': '09:15:01',
        'end_time' : '15:29:00',
    },
    'alpha': {
        'sigmult' : 0.5,
        'smoother' : {
            'enabled' : False,
            #'ul' : {'type' : 'WHIT' , 'lmbda' : 0.3, 'qlen' : 10 },            
            'ul' : {'type' : 'EMA' , 'timeperiod' : 0 },
            'cp' : {'type' : 'EMA' , 'timeperiod' : 0 },            
            'atm' : {'type' : 'STABLEWIN' , 'timeperiod' : 2 * 60 },
            'ts' : {'type' : 'STABLEWIN' , 'timeperiod' : 2 * 60 },            
        }
    },
    'execution': {
        'exec_instr_type' : 'NK_F',        
        'price_type' : 'LTP',
        'stop_loss': 0,
        'max_loss' : 0,
    }
}

In [ ]:
dump_base_dir = "/home/qoptrprod/data/dumps/"
underlyings = ["NIFTY_I","BANKNIFTY_I","FINNIFTY_I","MIDCPNIFTY_I"]
trading_dates = ["20240319", "20240320", "20240321", "20240322"]

In [ ]:
qoptr_results = []
scale_results = []
for underlying in underlyings:
    for trading_date in trading_dates:
        ul = underlying.split('_')[0]
        url = f'http://someurl.com/api/chain_app/runcmpbt/?ul={ul}&sdate={trading_date}&edate={trading_date}'
        response = requests.get(url=url)  
        qoptr_dict_list = json.loads(response.text)
        qoptr_day_result = qoptr_dict_list[0]['day_results'][0]
        qoptr_signals = qoptr_day_result['signals']    
        qdr = {}
        for k,v in qoptr_day_result['summary'].items():
            qdr[k] = v
        qdr['a.date'] = qoptr_day_result['date']
        qdr['b.underlying'] = qoptr_day_result['underlying']        
        qdr['expiry'] = qoptr_day_result['expiry']
        qoptr_results.append(qdr)

        sdr = {}
        arg_obj = ScaleBacktestArgGen.resolve_and_gen_args(dump_base_dir,underlying,trading_date=trading_date,bt_args=BtArgs)
        bt = ScaleBacktest(arg_obj)
        scale_result = bt.run_bt_with_qoptr_signal(qoptr_signals, shift=1)
        for k,v in scale_result['summary'].items():
            sdr[k] = v
        sdr['a.date'] = trading_date            
        sdr['b.underlying'] = underlying
        scale_results.append(sdr)

In [ ]:
qdf = pd.DataFrame(qoptr_results, columns=sorted(qoptr_results[0].keys()))
sdf = pd.DataFrame(scale_results, columns=sorted(scale_results[0].keys()))

In [ ]:
print(f"QOPTR Results: Total PNL: {round(qdf.net_pnl.sum())}")
display(qdf)
print(f"SCALE Results: Total PNL: {round(sdf.net_pnl.sum())}")
display(sdf)